Import Statements

In [73]:
import cv2
from otsu_helper import *
%matplotlib inline
import cv2
import matplotlib.pyplot as plt
import numpy as np

Load the Image Set

In [74]:
imageSet = "miku2"
raw_input_image, grey_input_image = loadImages(imageSet)
raw_input_image.shape

(716, 717, 3)

Perform Image Segmentation Using RGB values

In [75]:
# split the raw input image into 3 individual channels
blueChannel, greenChannel, redChannel = cv2.split(raw_input_image)

# apply segmentation on each individual channel
blueChannel_mask = minWithinClassVariance(blueChannel)
greenChannel_mask = minWithinClassVariance(greenChannel)
redChannel_mask = minWithinClassVariance(redChannel)

# # combine all masks together


  0%|          | 0/242 [00:00<?, ?it/s]/home/jo_wang/anaconda3/envs/ece661/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3715: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/jo_wang/anaconda3/envs/ece661/lib/python3.8/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/home/jo_wang/anaconda3/envs/ece661/lib/python3.8/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 256/256 [00:37<00:00,  6.85it/s]


In [76]:
mask_all = np.ones(blueChannel.shape)
mask_all = np.logical_and(mask_all, blueChannel_mask)
mask_all = np.logical_and(mask_all, redChannel_mask)
mask_all = np.logical_and(mask_all, greenChannel_mask).astype(np.uint8) * 255

cv2.imwrite(f"/home/jo_wang/Desktop/ECE661/HW06/rgb_segmentation/{imageSet}_rgb_seg.jpg", mask_all)
# cv2.imshow('maskall.jpg', mask_all)
# cv2.imshow('blue.jpg', blueChannel_mask)
# cv2.imshow('green.jpg', greenChannel_mask)
# cv2.imshow('red.jpg', redChannel_mask)
#
# cv2.waitKey(0)
# cv2.destroyAllWindows()

True

Perform Texture Based Segmentation

In [77]:
ch1 = performTexture(grey_input_image, 3)
ch2 = performTexture(grey_input_image, 5)
ch3 = performTexture(grey_input_image, 7)

ch1_otsu = minWithinClassVariance(ch1)
ch2_otsu = minWithinClassVariance(ch2)
ch3_otsu = minWithinClassVariance(ch3)

texture_all = np.ones(ch1_otsu.shape)
texture_all = np.logical_and(texture_all, ch1_otsu)
texture_all = np.logical_and(texture_all, ch2_otsu)

cv2.imwrite(f"/home/jo_wang/Desktop/ECE661/HW06/texture_segmentation/{imageSet}_texture_seg.jpg", np.logical_not(texture_all).astype(np.uint8) * 255)


100%|██████████| 256/256 [00:34<00:00,  7.41it/s]


True

Extract Contours

In [78]:
texture_all = np.logical_not(texture_all).astype(np.uint8) * 255
contourImage = getContour(np.logical_not(texture_all // 255).astype(np.uint8))
cv2.imwrite(f"/home/jo_wang/Desktop/ECE661/HW06/contour_extraction/{imageSet}_contour_seg.jpg", np.logical_not(contourImage // 255).astype(np.uint8) * 255)

True

In [79]:
print(texture_all.shape)
print(contourImage.shape)
og_bgr = np.hstack((raw_input_image, cv2.cvtColor(mask_all, cv2.COLOR_GRAY2RGB)))
texture_contour = np.hstack((cv2.cvtColor(texture_all, cv2.COLOR_GRAY2RGB), cv2.cvtColor(np.logical_not(contourImage // 255).astype(np.uint8) * 255, cv2.COLOR_GRAY2RGB) ) )
final_image = np.vstack((og_bgr, texture_contour))

# original       bgr segmentation
# texture        contour extraction
cv2.imwrite(f"/home/jo_wang/Desktop/ECE661/HW06/final_subplots/{imageSet}_final_subplots.jpg", final_image)


(716, 717)
(716, 717)


True